In [23]:
from gensim.models import Doc2Vec as D2V
from gensim.models.doc2vec import TaggedDocument
from nltk import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from os import listdir
from os.path import isfile, join
from nltk import download as nltkDownload

In [4]:
from db_helper import DBHelper

In [13]:
nltkDownload('stopwords')

[nltk_data] Downloading package stopwords to /home/hmenn/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
db = DBHelper()
db.connect(user="root",passwd="Hasan5695*",db="cse496")
tweets = db.getTweets("denemeShort")
tweets.sort(key=lambda t: t.date)

In [7]:
print("ExampleSentName:",tweets[0].text)

ExampleSentName: Hayat bu, bir bakarsın herşey bir anda son bulur.
 Hayat bu, son dediğin an herşey yeniden can bulur..Şemsi Tebrizi


In [14]:
tokenizer = RegexpTokenizer(r'\w+')
stopword_set = set(stopwords.words("turkish"))

In [16]:
def nlp_clean(data):
    new_data = []
    for d in data:
        new_str = d.lower()
        dlist = tokenizer.tokenize(new_str)
        dlist = list(set(dlist).difference(stopword_set))
        new_data.append(dlist)
    return new_data

In [20]:
cleanData = nlp_clean([tweet.text for tweet in tweets])

In [19]:
class LabeledLineSentence(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
              yield gensim.models.doc2vec.LabeledSentence(doc,[self.labels_list[idx]])

In [33]:
taggedData = [TaggedDocument(words=word_tokenize(_d.lower()), tags=str(i)) for i, _d in enumerate([tweet.text for tweet in tweets])]

In [34]:
print(taggedData[0])

TaggedDocument(['hayat', 'bu', ',', 'bir', 'bakarsın', 'herşey', 'bir', 'anda', 'son', 'bulur', '.', 'hayat', 'bu', ',', 'son', 'dediğin', 'an', 'herşey', 'yeniden', 'can', 'bulur..şemsi', 'tebrizi'], 0)


In [37]:
max_epochs = 10
vec_size = 20
alpha = 0.025

model = D2V(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.025,
                min_count=1,
                dm =1)
  
model.build_vocab(taggedData)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(taggedData,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("../outputs/d2v.model")
print("Model Saved")

iteration 0


/home/hmenn/Workspace/doc2graph/venv/lib/python3.5/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
Model Saved


In [38]:
model.docvecs.most_similar("1")

[('7', 0.9380130171775818),
 ('0', 0.7842957973480225),
 ('9', 0.7634994983673096),
 ('6', 0.7404583692550659),
 ('3', 0.703612208366394),
 ('2', 0.6429765224456787),
 ('8', 0.5338811874389648),
 ('4', 0.4151258170604706),
 ('5', 0.23789331316947937)]

In [39]:
tweets[1].text

'Kibir, bele bağlanmış taş gibidir. Onunla ne yüzülür ne de uçulur. (Hacı Bayram-ı Veli)'

In [41]:
tweets[0].text

'Hayat bu, bir bakarsın herşey bir anda son bulur.\n Hayat bu, son dediğin an herşey yeniden can bulur..Şemsi Tebrizi'